# Resiliência da Paisagem

## Importando pacotes e inicializando *geemap*

In [1]:
import os
import ee
import geemap

In [2]:
geemap.ee_initialize()

In [ ]:
A resiliência da paisagem é a média da diversidade e connectividade da paisagem.

## Base de Dados

Os dados para a resiliência da paisagem são o mapa de diversidade da paisagem e *Z-score* da connectividade criados anteriormente.

## Códigos para a criação da resiliência da paisagem

As análises foram rodadas no *Google Earth Engine* (Gorelick 2017), devido a demanda computacional do projeto, usando o pacote **geemap** (Wu 2020) em *Python* (Python Software Foundation 2023) como interface pela facilidade na documentação e reprodutividade das análises. 

Primeiro, importamos o polígono de biomas do Brasil para extraímos as coordenadas geográficas mínimas e máximas. Extraímos também a escala do modelo digital de elevação que estamos usando na análises prévias.

In [4]:
# Importando o polígono de bioma para definir as 
# coordenadas máximas e mínimas do Brasil

bioma = ee.FeatureCollection("projects/ee-lucasljardim9/assets/Biome")

# ModeloDigital de Elevação para extrair a resolução
DEM = ee.Image("MERIT/DEM/v1_0_3")

# função para extrair as bordas dos polígonos
def func_cmp(feature):
    return feature.bounds() 

# Extraindo as coordenadas mínimas e máximas do Brasil
bioma_box = bioma.map(func_cmp).geometry().dissolve(**{'maxError': 1}).bounds()

# Resolução das análises
escala = DEM.projection().nominalScale()

Importamos os mapas de diversidade da paisagem e *Z-score* da connectividade e calculamos a média entre eles.

In [7]:
Z_connectedness = ee.Image("projects/ee-lucasljardim9/assets/Z_connectedness_byregion")

Z_landscape_diversity = ee.Image("projects/ee-lucasljardim9/assets/landscape_diversity_byregion")

resilience = Z_connectedness.add(Z_landscape_diversity).divide(2)

Exportamos o mapa de resiliência como um *asset* no *Google Earth Engine*.

In [6]:
# Exporte o raster de diversidade da paisagem
assetId = "projects/ee-lucasljardim9/assets/resilience_byregion"

geemap.ee_export_image_to_asset(
    resilience, 
    description='resilience_byregion', 
    assetId=assetId, 
    region=bioma_box, 
    scale=escala,
    maxPixels=1e13
)